In [2]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost3	 = Beta('Cost3',0,None,None,0)
Cost4	 = Beta('Cost4',0,None,None,0)
Total_TT	 = Beta('Total_TT',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT * dur_walking
Opt2 = Constant2 + Total_TT * dur_cycling
Opt3 = Constant3 + Cost3 * cost_public + Total_TT * dur_public
Opt4 = Constant4 + Cost4 * cost_driving + Total_TT * dur_driving
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model1_cost"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

           Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2  -3.82    0.104   -36.9        0          0.11        -34.7   
Constant3  -0.86   0.0617   -13.9        0        0.0652        -13.2   
Constant4  -1.17   0.0739   -15.8        0        0.0824        -14.2   
Cost3     0.0779   0.0252    3.09    0.002        0.0269          2.9   
Cost4     -0.244   0.0197   -12.4        0        0.0251        -9.71   
Total_TT    -5.5    0.186   -29.6        0         0.216        -25.5   

           Rob. p-value  
Constant2             0  
Constant3             0  
Constant4             0  
Cost3           0.00372  
Cost4                 0  
Total_TT              0  
Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -4491.999
rho bar square = 0.351
Output file: logit_lpmc_model1_cost.html
